## Feature Enrichment

### Historical data enrichment

Pick one client (Site, aka sender_BIC) to do the enrichment as every site will be the same process

In [5]:
site_input_dir = "./processed_data"
site_name = "HCBHSGSG_Bank_9"

In [6]:
import os
import random
import string

import pandas as pd

history_file_name = os.path.join(site_input_dir, site_name, "history.csv")
df_history = pd.read_csv(history_file_name)
df_history

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,...,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label,Sender_BIC,Receiver_BIC,Currency,Beneficiary_BIC,Currency_Country
0,TXN_31639,USER_7709,257.68,POS,2023-01-01 01:06:00,61383.97,Tablet,Tokyo,Groceries,0,...,1099.31,Password,0.8859,0,1,HCBHSGSG,FBSFCHZH,INR,YSYCESMM,Mumbai
1,TXN_39239,USER_4475,99.42,ATM Withdrawal,2023-01-01 01:11:00,88959.44,Laptop,Tokyo,Travel,0,...,3412.79,Biometric,0.0152,0,1,HCBHSGSG,FBSFCHZH,JPY,HCBHSGSG,Tokyo
2,TXN_25673,USER_6728,49.57,Online,2023-01-01 01:13:00,96771.99,Laptop,Tokyo,Groceries,0,...,4104.06,Password,0.6304,1,1,HCBHSGSG,YMNYFRPP,USD,XITXUS33,New York
3,TXN_42035,USER_6935,179.96,POS,2023-01-01 02:17:00,77053.09,Mobile,Tokyo,Clothing,0,...,3412.87,OTP,0.7885,0,0,HCBHSGSG,YMNYFRPP,USD,ZHSZUS33,New York
4,TXN_22339,USER_6566,57.20,ATM Withdrawal,2023-01-01 02:25:00,54934.85,Laptop,Tokyo,Clothing,0,...,2728.42,PIN,0.7521,0,1,HCBHSGSG,FBSFCHZH,USD,XITXUS33,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5612,TXN_37621,USER_5430,117.85,Online,2023-07-20 19:14:00,45831.27,Laptop,Tokyo,Electronics,0,...,3833.14,OTP,0.3042,0,1,HCBHSGSG,XITXUS33,USD,XITXUS33,New York
5613,TXN_11380,USER_4892,91.13,Online,2023-07-20 20:18:00,27299.21,Mobile,Tokyo,Groceries,0,...,3099.80,OTP,0.1203,0,1,HCBHSGSG,SHSHKHH1,GBP,YXRXGB22,London
5614,TXN_43130,USER_3308,165.04,POS,2023-07-20 20:27:00,90191.79,Mobile,Tokyo,Travel,0,...,3092.85,OTP,0.1452,0,0,HCBHSGSG,YMNYFRPP,JPY,HCBHSGSG,Tokyo
5615,TXN_12293,USER_6058,12.32,ATM Withdrawal,2023-07-20 22:27:00,12385.73,Mobile,Tokyo,Electronics,0,...,854.99,OTP,0.6103,1,1,HCBHSGSG,ZHSZUS33,USD,ZHSZUS33,New York


In [7]:
history_summary = (
    df_history.groupby("Currency")
    .agg(
        hist_trans_volume=("Transaction_ID", "count"),
        hist_total_amount=("Transaction_Amount", "sum"),
        hist_average_amount=("Transaction_Amount", "mean"),
    )
    .reset_index()
)

history_summary

,Currency,hist_trans_volume,hist_total_amount,hist_average_amount
0,AUD,1145,112754.07,98.475170
1,GBP,1056,104216.90,98.690246
2,INR,1145,114992.08,100.429764
3,JPY,1140,115504.34,101.319596
4,USD,1131,108775.02,96.175968


# Enrich Feature with Currency

In [8]:
import pandas as pd

dataset_names = ["train", "test"]
results = {}

temp_ds_df = {}
temp_resampled_df = {}


for ds_name in dataset_names:
    file_name = os.path.join(site_input_dir, site_name, f"{ds_name}.csv")
    ds_df = pd.read_csv(file_name)
    ds_df["Timestamp"] = pd.to_datetime(ds_df["Timestamp"])

    # Set the Time column as the index
    ds_df.set_index("Timestamp", inplace=True)

    resampled_df = (
        ds_df.resample("1H")
        .agg(
            trans_volume=("Transaction_ID", "count"),
            total_amount=("Transaction_Amount", "sum"),
            average_amount=("Transaction_Amount", "mean"),
        )
        .reset_index()
    )

    temp_ds_df[ds_name] = ds_df
    temp_resampled_df[ds_name] = resampled_df

/tmp/ipykernel_24583/1206980181.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ds_df.resample("1H")
/tmp/ipykernel_24583/1206980181.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ds_df.resample("1H")


In [9]:
for ds_name in dataset_names:

    ds_df = temp_ds_df[ds_name]
    resampled_df = temp_resampled_df[ds_name]

    c_df = ds_df[["Currency"]].resample("1H").agg({"Currency": "first"}).reset_index()
    # Add Currency_Country to the resampled data by joining with the original DataFrame
    resampled_df2 = pd.merge(resampled_df, c_df, on="Timestamp")
    resampled_df3 = pd.merge(resampled_df2, history_summary, on="Currency")
    resampled_df4 = resampled_df3.copy()
    resampled_df4["x2_y1"] = (
        resampled_df4["average_amount"] / resampled_df4["hist_trans_volume"]
    )

    ds_df = ds_df.sort_values("Timestamp")
    resampled_df4 = resampled_df4.sort_values("Timestamp")

    merged_df = pd.merge_asof(ds_df, resampled_df4, on="Timestamp")
    merged_df = merged_df.drop(columns=["Currency_y"]).rename(
        columns={"Currency_x": "Currency"}
    )

    results[ds_name] = merged_df

print(results)

{'train':                Timestamp  Fraud_Label Transaction_ID    User_ID  \
0    2023-07-21 00:39:00            0      TXN_15663  USER_7774   
1    2023-07-21 00:40:00            0      TXN_16666  USER_2491   
2    2023-07-21 01:16:00            0      TXN_13793  USER_7209   
3    2023-07-21 02:21:00            0      TXN_33256  USER_9146   
4    2023-07-21 04:37:00            0      TXN_49460  USER_6842   
...                  ...          ...            ...        ...   
3584 2023-12-31 20:35:00            1      TXN_37832  USER_5830   
3585 2023-12-31 21:38:00            1      TXN_38974  USER_4134   
3586 2023-12-31 21:52:00            0      TXN_25394  USER_8356   
3587 2023-12-31 21:55:00            0      TXN_41124  USER_7621   
3588 2023-12-31 22:35:00            0      TXN_10811  USER_1721   

      Transaction_Amount Transaction_Type  Account_Balance Device_Type  \
0                 183.33   ATM Withdrawal         57770.62      Tablet   
1                  30.14    Bank Tran

/tmp/ipykernel_24583/3196226655.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  c_df = ds_df[["Currency"]].resample("1H").agg({"Currency": "first"}).reset_index()
/tmp/ipykernel_24583/3196226655.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  c_df = ds_df[["Currency"]].resample("1H").agg({"Currency": "first"}).reset_index()


# Enrich feature for beneficiary country

In [10]:
history_summary2 = (
    df_history.groupby("Beneficiary_BIC")
    .agg(
        hist_trans_volume=("Transaction_ID", "count"),
        hist_total_amount=("Transaction_Amount", "sum"),
        hist_average_amount=("Transaction_Amount", "mean"),
    )
    .reset_index()
)

history_summary2

,Beneficiary_BIC,hist_trans_volume,hist_total_amount,hist_average_amount
0,HCBHSGSG,1140,115504.34,101.319596
1,XITXUS33,555,54129.23,97.530144
2,YSYCESMM,1145,114992.08,100.429764
3,YXRXGB22,1056,104216.90,98.690246
4,ZHSZUS33,576,54645.79,94.871163
5,ZNZZAU3M,1145,112754.07,98.475170


In [11]:
import pandas as pd

dataset_names = ["train", "test"]
results2 = {}
for ds_name in dataset_names:
    ds_df = temp_ds_df[ds_name]
    resampled_df = temp_resampled_df[ds_name]

    c_df = (
        ds_df[["Beneficiary_BIC"]]
        .resample("1H")
        .agg({"Beneficiary_BIC": "first"})
        .reset_index()
    )

    # Add Beneficiary_BIC to the resampled data by joining with the original DataFrame
    resampled_df2 = pd.merge(resampled_df, c_df, on="Timestamp")

    resampled_df3 = pd.merge(resampled_df2, history_summary2, on="Beneficiary_BIC")

    resampled_df4 = resampled_df3.copy()
    resampled_df4["x3_y2"] = (
        resampled_df4["average_amount"] / resampled_df4["hist_trans_volume"]
    )

    ds_df = ds_df.sort_values("Timestamp")
    resampled_df4 = resampled_df4.sort_values("Timestamp")

    merged_df2 = pd.merge_asof(ds_df, resampled_df4, on="Timestamp")
    merged_df2 = merged_df2.drop(columns=["Beneficiary_BIC_y"]).rename(
        columns={"Beneficiary_BIC_x": "Beneficiary_BIC"}
    )

    results2[ds_name] = merged_df2

print(results2)

{'train':                Timestamp  Fraud_Label Transaction_ID    User_ID  \
0    2023-07-21 00:39:00            0      TXN_15663  USER_7774   
1    2023-07-21 00:40:00            0      TXN_16666  USER_2491   
2    2023-07-21 01:16:00            0      TXN_13793  USER_7209   
3    2023-07-21 02:21:00            0      TXN_33256  USER_9146   
4    2023-07-21 04:37:00            0      TXN_49460  USER_6842   
...                  ...          ...            ...        ...   
3584 2023-12-31 20:35:00            1      TXN_37832  USER_5830   
3585 2023-12-31 21:38:00            1      TXN_38974  USER_4134   
3586 2023-12-31 21:52:00            0      TXN_25394  USER_8356   
3587 2023-12-31 21:55:00            0      TXN_41124  USER_7621   
3588 2023-12-31 22:35:00            0      TXN_10811  USER_1721   

      Transaction_Amount Transaction_Type  Account_Balance Device_Type  \
0                 183.33   ATM Withdrawal         57770.62      Tablet   
1                  30.14    Bank Tran

/tmp/ipykernel_24583/2998991707.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("1H")
/tmp/ipykernel_24583/2998991707.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("1H")


In [12]:
final_results = {}
for name in results:
    df = results[name]
    df2 = results2[name]
    df3 = df2[["Timestamp", "Beneficiary_BIC", "x3_y2"]].copy()
    df4 = pd.merge(df, df3, on=["Timestamp", "Beneficiary_BIC"])
    final_results[name] = df4


for name in final_results:
    site_dir = os.path.join(site_input_dir, site_name)
    os.makedirs(site_dir, exist_ok=True)
    enrich_file_name = os.path.join(site_dir, f"{name}_enrichment.csv")
    print(enrich_file_name)
    final_results[name].to_csv(enrich_file_name)

final_results["train"]

./processed_data/HCBHSGSG_Bank_9/train_enrichment.csv
./processed_data/HCBHSGSG_Bank_9/test_enrichment.csv


,Timestamp,Fraud_Label,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Account_Balance,Device_Type,Location,Merchant_Category,...,Beneficiary_BIC,Currency_Country,trans_volume,total_amount,average_amount,hist_trans_volume,hist_total_amount,hist_average_amount,x2_y1,x3_y2
0,2023-07-21 00:39:00,0,TXN_15663,USER_7774,183.33,ATM Withdrawal,57770.62,Tablet,Tokyo,Restaurants,...,ZNZZAU3M,Sydney,2,213.47,106.735,1145,112754.07,98.475170,0.093218,0.093218
1,2023-07-21 00:40:00,0,TXN_16666,USER_2491,30.14,Bank Transfer,13256.87,Mobile,Tokyo,Electronics,...,YSYCESMM,Mumbai,2,213.47,106.735,1145,112754.07,98.475170,0.093218,0.093218
2,2023-07-21 01:16:00,0,TXN_13793,USER_7209,51.05,ATM Withdrawal,78156.21,Laptop,Tokyo,Clothing,...,HCBHSGSG,Tokyo,1,51.05,51.050,1140,115504.34,101.319596,0.044781,0.044781
3,2023-07-21 02:21:00,0,TXN_33256,USER_9146,59.16,Online,16293.82,Tablet,Tokyo,Clothing,...,YSYCESMM,Mumbai,1,59.16,59.160,1145,114992.08,100.429764,0.051668,0.051668
4,2023-07-21 04:37:00,0,TXN_49460,USER_6842,20.76,POS,9549.72,Laptop,Tokyo,Groceries,...,ZHSZUS33,New York,1,20.76,20.760,1131,108775.02,96.175968,0.018355,0.036042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,2023-12-31 20:35:00,1,TXN_37832,USER_5830,50.72,ATM Withdrawal,62311.01,Laptop,Tokyo,Groceries,...,YSYCESMM,Mumbai,1,50.72,50.720,1145,114992.08,100.429764,0.044297,0.044297
3597,2023-12-31 21:38:00,1,TXN_38974,USER_4134,85.95,Online,93524.52,Tablet,Tokyo,Electronics,...,YSYCESMM,Mumbai,3,89.61,29.870,1145,114992.08,100.429764,0.026087,0.026087
3598,2023-12-31 21:52:00,0,TXN_25394,USER_8356,1.38,ATM Withdrawal,6957.28,Laptop,Tokyo,Travel,...,ZNZZAU3M,Sydney,3,89.61,29.870,1145,114992.08,100.429764,0.026087,0.026087
3599,2023-12-31 21:55:00,0,TXN_41124,USER_7621,2.28,ATM Withdrawal,35539.86,Laptop,Tokyo,Travel,...,YXRXGB22,London,3,89.61,29.870,1145,114992.08,100.429764,0.026087,0.026087


Let's go back to the [XGBoost Notebook](../xgboost.ipynb)